# Creating a weather API

The website will need an API that can return a list of destinations on the specified CCAAs (or the whole country) on a given date based on the predicted weather.

It must meet the following criteria:

- Takes a date and derives the week of the year.
- Returns the 5 best destination on that date based on meteorological data.
- Filters the 5 destinations by the specified CCAAs.
- Spreads results between all selected CCAAs.

# Assigning a weather station to each destination

Since most destination towns don't have a weather station we'll be simply assigning them the closest one (for the moment). This will be done by creating a new dataframe containing all town IDs (one for each row) as well as the weekly index of the nearest weather station.

In [1]:
import pandas as pd

In [2]:
#Importing our towns dataframe.

towns = pd.read_csv('towns_2207_201.csv')

In [3]:
towns.head()

,ID,municipality,ccaa,province,municipality_inhabitants,geographic_area,radius,routes_number,routes_ids,mountain_passes_ids,coords,coords_MDB
0,884,Barcelona,Cataluña,Barcelona,1664182,100.764400,5.663411,4,"[1292, 1732, 6228, 8149]","[315, 323, 822, 388]","(41.38424664,2.17634927)","[2.17634927,41.38424664]"
1,7257,València,Comunitat Valenciana,Valencia,800215,139.268700,6.658115,9,"[528, 1469, 1472, 2478, 5225, 7040, 7231, 7734...","[112, 631, 773, 1092, 597, 533, 769, 52, 424, ...","(39.47534441,-0.37565717)","[-0.37565717,39.47534441]"
2,4547,Málaga,Andalucía,Málaga,578460,395.706900,11.223062,10,"[2933, 4541, 4546, 5035, 5997, 5998, 8379, 841...","[86, 108, 109, 649, 171, 426, 488, 688, 700, 1...","(36.72034267,-4.41997511)","[-4.41997511,36.72034267]"
3,4613,Murcia,Región de Murcia,Murcia,459403,885.114900,16.785117,7,"[691, 1691, 6089, 6769, 7049, 8099, 8196]","[11, 787, 1103, 1055, 1012, 978, 1001]","(37.98436361,-1.1285408)","[-1.1285408,37.98436361]"
4,151,Alicante,Comunitat Valenciana,Alicante,337482,201.265845,8.004046,9,"[621, 629, 647, 680, 1469, 3189, 8145, 9500, 9...","[956, 626, 643, 644, 429, 534, 802, 99, 1092, ...","(38.34548705,-0.4831832)","[-0.4831832,38.34548705]"


In [4]:
#Importing our weather station data.

meteo = pd.read_csv('meteo.csv')

In [7]:
meteo.head()

,1,2,3,4,5,6,7,8,9,10,...,46,47,48,49,50,51,52,53,name,coords
0,7.8,7.4,7.4,7.4,7.6,7.8,8.0,8.0,8.0,8.4,...,7.8,8.0,7.8,8.0,7.8,8.0,8.0,8.0,VANDELL�S,"(41.02154, 0.83029)"
1,7.8,7.4,7.4,7.4,7.8,7.8,8.0,8.0,7.8,8.6,...,8.2,7.8,7.8,7.8,7.8,8.0,7.8,7.4,REUS AEROPUERTO,"(41.145,1.16361)"
2,7.8,7.4,7.4,6.2,7.8,7.8,7.8,7.8,7.8,7.8,...,8.2,7.8,7.8,7.8,7.8,7.8,7.8,7.8,BARCELONA AEROPUERTO,"(41.29278,2.07)"
3,7.6,7.2,7.6,6.4,7.6,7.6,8.0,8.0,8.0,8.0,...,8.0,7.6,7.6,7.6,7.6,7.6,7.6,7.2,MANRESA,"(41.72,1.84028)"
4,7.4,7.4,7.4,6.6,7.4,7.4,7.4,7.8,7.8,7.8,...,7.0,7.8,7.4,7.0,7.4,7.4,7.4,7.4,"BARCELONA, FABRA","(41.41833,2.12417)"


In [14]:
#Creating a new dataframe with the same columns as meteo.

df = pd.DataFrame(columns=meteo.columns)

In [15]:
df.head()

,1,2,3,4,5,6,7,8,9,10,...,46,47,48,49,50,51,52,53,name,coords


In [18]:
#Adding new columns to hold town ID,  CCAA and coords.

df['town_ID'] = None
df['CCAA_ID'] = None
df['coords'] = None

In [20]:
#Let's popuate the town_ID column by using a list.

df['town_ID'] = towns['ID'].tolist()

In [23]:
#Doing the same with CCAA.

df['CCAA_ID'] = towns['ccaa'].tolist()

In [28]:
#Performing the same procedure with the coordinates.

df['coords'] = towns['coords'].tolist()

In [29]:
df.head()

,1,2,3,4,5,6,7,8,9,10,...,48,49,50,51,52,53,name,coords,town_ID,CCAA_ID
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(41.38424664,2.17634927)",884,Cataluña
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(39.47534441,-0.37565717)",7257,Comunitat Valenciana
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(36.72034267,-4.41997511)",4547,Andalucía
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(37.98436361,-1.1285408)",4613,Región de Murcia
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(38.34548705,-0.4831832)",151,Comunitat Valenciana


In [25]:
df['CCAA_ID'].value_counts()

Andalucía               44
Comunitat Valenciana    27
País Vasco              20
Galicia                 15
Asturias                13
Cantabria               10
Cataluña                 9
Aragón                   8
Navarra                  7
Extremadura              5
Comunidad de Madrid      5
Castilla-La Mancha       4
Región de Murcia         3
La Rioja                 2
Name: CCAA_ID, dtype: int64

In [ ]:
#Our next step will be turning each CCAA name into its ID this will have to be performed with a loop.

for i in range(len(df)):
    if df['CCAA_ID'].iloc[i] == ''